In [15]:
import pandas as pd 
import numpy as np 
import configparser
import os
import seaborn as sns
import plotly_express as px
import kaleido

import nltk 

config = configparser.ConfigParser()
config.read("../env.ini")
data_home = config['DEFAULT']['data_home']
output_dir = config['DEFAULT']['output_dir']
base_path = config['DEFAULT']['base_path']
code_dir = config['DEFAULT']['code_dir']

data_prefix = 'Maha'
out_path = f'{output_dir}/{data_prefix}'
OHCO = ['book_id','chap_id','sec_id','para_num', 'sent_num', 'token_num']


os.chdir(code_dir)
import preprocess
import bow_analysis
os.chdir(base_path + "\\notebooks")

In [16]:
CORPUS = pd.read_csv(rf"{output_dir}\F2\{data_prefix}-CORPUS.csv", sep="|").set_index(OHCO)
LIB = pd.read_csv(fr"{output_dir}\F2\{data_prefix}-LIB.csv", sep="|").set_index('book_id')
LIB2 = pd.read_csv(fr"{output_dir}\F2\{data_prefix}-LIB2.csv", sep="|").set_index(OHCO[:2])
VOCAB = pd.read_csv(rf"{output_dir}\F2\{data_prefix}-VOCAB.csv", sep="|").set_index("term_str")
VOCAB2 = pd.read_csv(rf"{output_dir}\CHAP_BOW\{data_prefix}-VOCAB2.csv", sep="|").set_index("term_str")
TFIDF = pd.read_csv(rf"{output_dir}\CHAP_BOW\{data_prefix}-TFIDF_REDUCED_CHAPS_L2.csv", sep="|").set_index(OHCO[:2])

In [17]:
TFIDF

shafts   pierced      army   deities  carwarriors  \
book_id chap_id                                                        
1       1        0.000000  0.017978  0.029045  0.000000     0.000000   
        2        0.000000  0.006462  0.037584  0.009168     0.000000   
        3        0.000000  0.000000  0.000000  0.013734     0.000000   
        4        0.000000  0.009420  0.000000  0.000000     0.000000   
        5        0.000000  0.020106  0.000000  0.032600     0.000000   
...                   ...       ...       ...       ...          ...   
15      2        0.000000  0.000000  0.000000  0.226051     0.013851   
        3        0.017530  0.000000  0.000000  0.000000     0.000000   
16      1        0.062153  0.014705  0.000000  0.041726     0.086291   
17      1        0.000000  0.000000  0.000000  0.201742     0.000000   
18      1        0.000000  0.000000  0.000000  0.296294     0.059203   

                   troops   sanjaya       cut  penances    partha  ...  \
book_id chap_id                                                    ...   
1       1        0.000000  0.674489  0.000000  0.000000  0.000000  ...   
        2        0.050112  0.049816  0.000000  0.000000  0.000000  ...   
        3        0.000000  0.000000  0.000000  0.000000  0.000000  ...   
        4        0.000000  0.000000  0.000000  0.022543  0.000000  ...   
        5        0.000000  0.000000  0.016690  0.099664  0.000000  ...   
...                   ...       ...       ...       ...       ...  ...   
15      2        0.030889  0.000000  0.000000  0.317740  0.000000  ...   
        3        0.000000  0.374090  0.000000  0.111660  0.000000  ...   
16      1        0.014255  0.000000  0.000000  0.035190  0.058821  ...   
17      1        0.000000  0.000000  0.000000  0.042535  0.017775  ...   
18      1        0.000000  0.000000  0.010552  0.043458  0.000000  ...   

                 propitiatory  apartment  anguish    thorn  cheered  \
book_id chap_id                                                       
1       1                 0.0   0.000000  0.01683  0.00000      0.0   
        2                 0.0   0.000000  0.00000  0.00000      0.0   
        3                 0.0   0.000000  0.00000  0.00000      0.0   
        4                 0.0   0.054643  0.00000  0.00000      0.0   
        5                 0.0   0.000000  0.00000  0.00833      0.0   
...                       ...        ...      ...      ...      ...   
15      2                 0.0   0.000000  0.00000  0.00000      0.0   
        3                 0.0   0.000000  0.00000  0.00000      0.0   
16      1                 0.0   0.000000  0.00000  0.00000      0.0   
17      1                 0.0   0.000000  0.00000  0.00000      0.0   
18      1                 0.0   0.000000  0.00000  0.00000      0.0   

                 believing     tops  pressing       hid    affair  
book_id chap_id                                                    
1       1              0.0  0.00000   0.00000  0.000000  0.000000  
        2              0.0  0.00000   0.00000  0.000000  0.000000  
        3              0.0  0.00000   0.00000  0.000000  0.000000  
        4              0.0  0.00000   0.00000  0.000000  0.027321  
        5              0.0  0.00833   0.00833  0.008066  0.000000  
...                    ...      ...       ...       ...       ...  
15      2              0.0  0.00000   0.00000  0.000000  0.000000  
        3              0.0  0.00000   0.00000  0.000000  0.000000  
16      1              0.0  0.00000   0.00000  0.000000  0.000000  
17      1              0.0  0.00000   0.00000  0.000000  0.000000  
18      1              0.0  0.00000   0.00000  0.000000  0.000000  

[99 rows x 3080 columns]

In [18]:
from sklearn.decomposition import PCA
n_comps = 10


pc_cols = [f"PC{i}" for i in range(n_comps)]
pca_engine = PCA(n_components=n_comps)
DCM = pd.DataFrame(pca_engine.fit_transform(TFIDF), index=TFIDF.index)
DCM.columns = ['PC{}'.format(i) for i in DCM.columns]
DCM

PC0       PC1       PC2       PC3       PC4       PC5  \
book_id chap_id                                                               
1       1        0.015681 -0.036521 -0.298223  0.177523  0.182230  0.519846   
        2       -0.105805  0.035676 -0.023534  0.029437  0.070474  0.117260   
        3       -0.173519  0.064475  0.077054 -0.083255 -0.052395  0.105664   
        4       -0.181256  0.190198  0.270879  0.251735  0.025256  0.065040   
        5       -0.150659 -0.040650  0.159117 -0.078535 -0.118136  0.028463   
...                   ...       ...       ...       ...       ...       ...   
15      2       -0.179516 -0.258161  0.001110  0.080537  0.231420 -0.150905   
        3       -0.052521 -0.109680 -0.251249  0.219385  0.266187  0.115013   
16      1        0.008625  0.012303  0.007870 -0.156257  0.253453 -0.237248   
17      1       -0.147089 -0.081836  0.062155 -0.066898  0.202896 -0.034807   
18      1       -0.193043 -0.313593  0.146701 -0.067164  0.144590 -0.025664   

                      PC6       PC7       PC8       PC9  
book_id chap_id                                          
1       1       -0.110690 -0.101600 -0.144417 -0.180085  
        2       -0.076553 -0.053089  0.003346 -0.088018  
        3       -0.026661 -0.008051  0.085232 -0.042655  
        4        0.026060 -0.038584 -0.042887 -0.090358  
        5        0.016918 -0.136649  0.038219 -0.115667  
...                   ...       ...       ...       ...  
15      2       -0.288002  0.079650  0.185364 -0.116158  
        3       -0.347073 -0.040921  0.173923 -0.124253  
16      1       -0.275734 -0.068821 -0.131297  0.286885  
17      1       -0.128074  0.065899  0.088601  0.093766  
18      1       -0.093676  0.087959  0.007013 -0.091208  

[99 rows x 10 columns]

In [19]:
DCM = DCM.join(LIB2[['book_name','chap_name']], on=OHCO[:2])
DCM = DCM.join(LIB, on=OHCO[:1])
DCM

PC0       PC1       PC2       PC3       PC4       PC5  \
book_id chap_id                                                               
1       1        0.015681 -0.036521 -0.298223  0.177523  0.182230  0.519846   
        2       -0.105805  0.035676 -0.023534  0.029437  0.070474  0.117260   
        3       -0.173519  0.064475  0.077054 -0.083255 -0.052395  0.105664   
        4       -0.181256  0.190198  0.270879  0.251735  0.025256  0.065040   
        5       -0.150659 -0.040650  0.159117 -0.078535 -0.118136  0.028463   
...                   ...       ...       ...       ...       ...       ...   
15      2       -0.179516 -0.258161  0.001110  0.080537  0.231420 -0.150905   
        3       -0.052521 -0.109680 -0.251249  0.219385  0.266187  0.115013   
16      1        0.008625  0.012303  0.007870 -0.156257  0.253453 -0.237248   
17      1       -0.147089 -0.081836  0.062155 -0.066898  0.202896 -0.034807   
18      1       -0.193043 -0.313593  0.146701 -0.067164  0.144590 -0.025664   

                      PC6       PC7       PC8       PC9  ... n_chaps n_secs  \
book_id chap_id                                          ...                  
1       1       -0.110690 -0.101600 -0.144417 -0.180085  ...      19    235   
        2       -0.076553 -0.053089  0.003346 -0.088018  ...      19    235   
        3       -0.026661 -0.008051  0.085232 -0.042655  ...      19    235   
        4        0.026060 -0.038584 -0.042887 -0.090358  ...      19    235   
        5        0.016918 -0.136649  0.038219 -0.115667  ...      19    235   
...                   ...       ...       ...       ...  ...     ...    ...   
15      2       -0.288002  0.079650  0.185364 -0.116158  ...       3     38   
        3       -0.347073 -0.040921  0.173923 -0.124253  ...       3     38   
16      1       -0.275734 -0.068821 -0.131297  0.286885  ...       1      7   
17      1       -0.128074  0.065899  0.088601  0.093766  ...       1      2   
18      1       -0.093676  0.087959  0.007013 -0.091208  ...       1      5   

                n_sents n_tokens  n_chars       parva_name  timeline  \
book_id chap_id                                                        
1       1         12422   228684  1055885              Adi   Pre-War   
        2         12422   228684  1055885              Adi   Pre-War   
        3         12422   228684  1055885              Adi   Pre-War   
        4         12422   228684  1055885              Adi   Pre-War   
        5         12422   228684  1055885              Adi   Pre-War   
...                 ...      ...      ...              ...       ...   
15      2          1742    30376   141565    Ashramavasika  Post-War   
        3          1742    30376   141565    Ashramavasika  Post-War   
16      1           432     8072    37596          Mausala  Post-War   
17      1           206     3062    14086  Mahaprasthanika  Post-War   
18      1           495     8628    40381     Svargarohana  Post-War   

                                                         key_focus  \
book_id chap_id                                                      
1       1            Origins, family lineages, childhood of heroes   
        2            Origins, family lineages, childhood of heroes   
        3            Origins, family lineages, childhood of heroes   
        4            Origins, family lineages, childhood of heroes   
        5            Origins, family lineages, childhood of heroes   
...                                                            ...   
15      2                           Retirement and death of elders   
        3                           Retirement and death of elders   
16      1              Destruction of Yadava clan, Krishna's death   
17      1           Pandavas renounce kingdom, begin final journey   
18      1        Yudhishthira’s ascent to heaven and cosmic res...   

                   main_theme   broad_theme  
book_id chap_id                              
1       1             line

In [20]:
DCM.to_csv(fr"{output_dir}\PCA_Results\{data_prefix}-PCA10_DCM.csv",sep = "|")

In [21]:
LOADINGS = pd.DataFrame(pca_engine.components_.T * np.sqrt(pca_engine.explained_variance_))
LOADINGS.index = TFIDF.columns
LOADINGS.columns = ["PC{}".format(i) for i in LOADINGS.columns]
LOADINGS.index.name = 'term_str'
LOADINGS2 = LOADINGS.join(VOCAB2, on="term_str")
LOADINGS2

,PC0,PC1,PC2,PC3,PC4,PC5,PC6,PC7,PC8,PC9,...,n_pos,cat_pos,df,idf,dp,di,dh,dfidf,tfidf_mean,tfidf_sum
term_str,,,,,,,,,,,,,,,,,,,,,
shafts,0.081493,-0.008034,0.016105,-0.005387,-0.007990,-0.006501,0.006141,-0.009424,0.008726,0.001449,...,11,"{'JJ', 'RB', 'VB', 'NNP', 'VBP', 'NN', 'VBD', ...",57.0,0.796467,0.575758,0.796467,0.458572,45.398597,0.000569,0.056298
pierced,0.050607,-0.006921,0.010534,-0.001614,-0.004611,-0.003369,0.006120,-0.004718,0.004614,-0.000154,...,5,"{'JJ', 'NNP', 'VBD', 'NN', 'VBN'}",55.0,0.847997,0.555556,0.847997,0.471109,46.639830,0.000360,0.035682
army,0.038257,-0.007304,-0.005195,0.000051,-0.002049,0.003423,0.005395,0.002014,-0.007463,-0.001635,...,8,"{'JJ', 'NNP', 'VBP', 'IN', 'NN', 'VBD', 'VBZ',...",56.0,0.822002,0.565657,0.822002,0.464971,46.032095,0.000389,0.038510
deities,-0.019501,-0.043414,0.020390,-0.008370,0.014691,-0.009879,-0.004952,0.010633,0.000027,-0.008770,...,5,"{'JJ', 'NNP', 'VBZ', 'NNPS', 'NNS'}",43.0,1.203092,0.434343,1.203092,0.522555,51.732950,0.000281,0.027864
carwarriors,0.054084,-0.008152,0.001389,0.002997,0.003492,-0.007961,0.001567,0.001163,0.000908,0.001593,...,2,"{'NNP', 'NNS'}",46.0,1.105795,0.464646,1.105795,0.513804,50.866555,0.000441,0.043627
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
believing,-0.000406,-0.000234,-0.000492,0.000590,-0.001145,-0.000400,0.000433,-0.000988,-0.000318,-0.000437,...,2,"{'NN', 'VBG'}",18.0,2.459432,0.181818,2.459432,0.447169,44.269769,0.000023,0.002284
tops,-0.000455,-0.000036,-0.001500,0.000411,-0.000384,0.000874,0.000156,-0.000527,0.000109,-0.000624,...,1,{'NNS'},18.0,2.459432,0.181818,2.459432,0.447169,44.269769,0.000036,0.003612
pressing,0.000154,0.003219,0.001076,0.003521,0.001688,-0.001485,0.002819,0.001598,0.000323,-0.001305,...,1,{'VBG'},18.0,2.459432,0.181818,2.459432,0.447169,44.269769,0.000055,0.005441


In [22]:
LOADINGS2.to_csv(fr"{output_dir}\PCA_Results\{data_prefix}-PCA10_LOADINGS.csv",sep = "|")

In [23]:
def vis_pcs(M, a, b,color ="book_name" ,hover_name='chap_name'):
    M["book_name"] = pd.Categorical(M["book_name"], ordered=True)
    return px.scatter(M, f"PC{a}", f"PC{b}", 
                      color = color ,
                    hover_name=hover_name, 
                    # symbol=symbol, size=size,
                    marginal_x='box', height=800, width = 1008)

def vis_loadings(LOADINGS, a=0, b=1, hover_name='term_str'):
    return px.scatter(LOADINGS.reset_index(), f"PC{a}", f"PC{b}", 
                      text='term_str', hover_name='term_str',
                       color='max_pos_group', size="dfidf",
                      marginal_x='box', height=800)

In [24]:
fig01pcs = vis_pcs(DCM, 0, 1, "timeline")
# fig01pcs.write_image(fr"{output_dir}\PCA_Results\{data_prefix}-PCA10_01PCS.png")
fig01pcs.show()

In [12]:
fig01load = vis_loadings(LOADINGS2, 0, 1)
fig01load.show()

In [19]:
fig12pcs = vis_pcs(DCM, 1, 2, "timeline")
fig12pcs.show()

In [14]:
fig12load = vis_loadings(LOADINGS2, 1, 2)
fig12load.show()

In [20]:
top_terms_sk= {}

data = []
for i in range(n_comps):
    for j in [0, 1]:
        data.append((f"PC{i}", j, ' '.join(LOADINGS.sort_values(f'PC{i}', ascending=bool(j)).head(10).index.to_list())))

comp_strs = pd.DataFrame(data)
comp_strs.columns =  ['pc', 'pole', 'top_terms']
comp_strs = comp_strs.set_index(['pc', 'pole'])


COMPS = comp_strs
COMPS['top_doc'] = pd.concat([DCM[pc_cols].idxmax(), DCM[pc_cols].idxmin()], axis=0, keys=[0,1])\
    .to_frame('top_doc')\
    .reset_index()\
    .rename(columns={'level_0':'pole', 'level_1':'pc'})\
    .set_index(['pc','pole'])\
    .sort_index()

In [21]:
COMPS

top_terms  top_doc
pc  pole                                                            
PC0 0     shafts carwarriors pierced sanjaya army cut dr...   (7, 6)
    1     deities vidura penances vyasa brahman becomes ...  (13, 1)
PC1 0     rakshasa jarasandha monarchs virata sisupala r...   (3, 2)
    1     deities righteousness becomes penances brahman...  (12, 3)
PC2 0     rakshasa deities shafts bhrigu rama markandeya...   (1, 9)
    1     sanjaya vidura gandhari dussasana hope dhritar...   (5, 2)
PC3 0     rakshasa vidura sanjaya gandhari hope sleeping...   (3, 2)
    1     jarasandha sisupala monarchs partha vrishni an...  (1, 18)
PC4 0     jarasandha rakshasa sanjaya magadha gandhari d...   (2, 3)
    1     virata dice spake play dussasana matsyas vidur...   (4, 3)
PC5 0     sanjaya island hope islands serpent yaksha att...   (1, 1)
    1     vidura rakshasa rama ladies gandhari drupada v...  (1, 14)
PC6 0     jarasandha rakshasa sisupala magadha monarchs ...   (2, 3)
    1     gandhari ladies rama sanjaya savitri vyasa and...  (15, 3)
PC7 0     virata matsyas jarasandha viratas uttara deiti...   (4, 1)
    1     vidura sanjaya mansion maya vasishtha history ...   (2, 2)
PC8 0     jarasandha gandhari penances vidura deities co...   (2, 4)
    1     rama andhakas vrishnis markandeya vrishni kesa...  (1, 18)
PC9 0     rakshasa virata sanjaya deities rama vyasa his...   (1, 1)
    1     jarasandha camp dronas vidura serpent misfortu...  (3, 17)

In [22]:
TFIDF.loc[(7, 6)].sort_values(ascending=False).head(20)

term_str
shafts         0.389722
satyaki        0.316299
pierced        0.219966
sindhus        0.217252
carwarriors    0.165149
cut            0.146215
sanjaya        0.145163
sini           0.144992
yuyudhana      0.133321
bhurisravas    0.123660
keen           0.118353
grandson       0.116777
shot           0.115106
satwata        0.108736
host           0.107483
rushed         0.104546
carwarrior     0.101800
showers        0.099959
partha         0.097485
kritavarman    0.096659
Name: (7, 6), dtype: float64

In [23]:
comp_strs.unstack()

top_terms  \
pole                                                  0   
pc                                                        
PC0   shafts carwarriors pierced sanjaya army cut dr...   
PC1   rakshasa jarasandha monarchs virata sisupala r...   
PC2   rakshasa deities shafts bhrigu rama markandeya...   
PC3   rakshasa vidura sanjaya gandhari hope sleeping...   
PC4   jarasandha rakshasa sanjaya magadha gandhari d...   
PC5   sanjaya island hope islands serpent yaksha att...   
PC6   jarasandha rakshasa sisupala magadha monarchs ...   
PC7   virata matsyas jarasandha viratas uttara deiti...   
PC8   jarasandha gandhari penances vidura deities co...   
PC9   rakshasa virata sanjaya deities rama vyasa his...   

                                                        top_doc           
pole                                                  1       0        1  
pc                                                                        
PC0   deities vidura penances vyasa brahman becomes ...  (7, 6)  (13, 1)  
PC1   deities righteousness becomes penances brahman...  (3, 2)  (12, 3)  
PC2   sanjaya vidura gandhari dussasana hope dhritar...  (1, 9)   (5, 2)  
PC3   jarasandha sisupala monarchs partha vrishni an...  (3, 2)  (1, 18)  
PC4   virata dice spake play dussasana matsyas vidur...  (2, 3)   (4, 3)  
PC5   vidura rakshasa rama ladies gandhari drupada v...  (1, 1)  (1, 14)  
PC6   gandhari ladies rama sanjaya savitri vyasa and...  (2, 3)  (15, 3)  
PC7   vidura sanjaya mansion maya vasishtha history ...  (4, 1)   (2, 2)  
PC8   rama andhakas vrishnis markandeya vrishni kesa...  (2, 4)  (1, 18)  
PC9   jarasandha camp dronas vidura serpent misfortu...  (1, 1)  (3, 17)

In [24]:
COMPS.to_csv(fr"{output_dir}\PCA_Results\{data_prefix}-PCA10_Components.csv",sep = "|")